In [ ]:
import sympy as sp 
from sympy import Derivative
from IPython.display import display, Math

from libs.display import *
from libs.utils_sympy import *
from libs.reconstruction_amr import * 
DEBUG = 0
CHRISTIAN_VERSION = 1
ORDER_SERIES = 4
DT_FROM_CFL = 0 # choisir si on défini dt à partir de la CFL ou si l'on défini la CFL à partir de dt. 
D = sp.Symbol("D", real=True) #Coefficient de diffusion
x, t = sp.symbols("x t", real=True)#Varaiables espace-temps
dx = sp.Symbol(r"\Delta x", real=True)#Pas de discrétisation spatial (niveau fin et grille uniforme)
u = sp.Function("u")#Fonction "régulière" vérifiant le schéma
if DT_FROM_CFL :
    CFL_DIFF = sp.Symbol(r"\lambda", real=True)
    dt = dx**2 * CFL_DIFF / D
else : 
    dt = sp.Symbol(r"\Delta t", real=True)
    CFL_DIFF = D * dt / dx**2
#dt = dx
CFL_DIFF = D * dt / dx**2 

In [ ]:
u_xplus1 = developpement_taylor_spatial(u,+dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x+dx
u_xplus2 = developpement_taylor_spatial(u,+2*dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x+2dx
u_xmoins1 = developpement_taylor_spatial(u,-dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x-dx
u_xmoins2 = developpement_taylor_spatial(u,-2*dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x-2dx
# Ce qui reste en plus de dt_u lorsque l'on développe en série (u^t+1 - u^t)/dt
reste_serie_en_temps =((developpement_taylor_temps(u,dt,ORDER_SERIES,x,t) - u(x,t))/dt - sp.diff(u(x,t),t)).expand()
# Schéma Volume fini Laplacien drordre 2 en temps et en espace
flux_total_sans_AMR = (1/dt*CFL_DIFF * (u_xplus1 - 2*u(x,t) + u_xmoins1) + 1/dt *  sp.Rational(1,2) * CFL_DIFF**2 * (u_xplus2 - 4*u_xplus1 + 6*u(x,t) - 4*u_xmoins1 + u_xmoins2)).expand()
u_t = - reste_serie_en_temps + flux_total_sans_AMR
u_t_tilde = u_t.expand()
u_t_tilde = apply_cauchy_kovaleskaya(u_t_tilde,u,x,t,D,ORDER_SERIES)
second_membre = rendreJoli(u_t_tilde,u,x,t,dx,dt,ORDER_SERIES)
derivee_temporelle = sp.diff(u(x, t), t)
#eq_sans_AMR = f"{sp.latex(derivee_temporelle)} = {second_membre}"
eq_sans_AMR = f"{sp.latex(derivee_temporelle)} = {sp.latex(u_t_tilde)}"
print("Eq modifée temps-espace RKE2")
#display(Math(eq_sans_AMR))
display(Math(eq_sans_AMR))
print(eq_sans_AMR)
## Christian 
flux_total_sans_AMR_Chr = (1/dt*CFL_DIFF * (u_xplus1 - 2*u(x,t) + u_xmoins1) + 1/dt *  sp.Rational(1,2) * CFL_DIFF*(CFL_DIFF - sp.Rational(1,6))* (u_xplus2 - 4*u_xplus1 + 6*u(x,t) - 4*u_xmoins1 + u_xmoins2)).expand()
u_t_Chr = - reste_serie_en_temps + flux_total_sans_AMR_Chr
u_t_tilde_Chr = u_t_Chr.expand()
u_t_tilde_Chr = apply_cauchy_kovaleskaya(u_t_tilde_Chr,u,x,t,D,ORDER_SERIES)
second_membre_Chr = rendreJoli(u_t_tilde_Chr,u,x,t,dx,dt,ORDER_SERIES)
eq_sans_AMR_Chr = f"{sp.latex(derivee_temporelle)} = {sp.latex(u_t_tilde_Chr)}"
print("Eq modifée temps-espace Version Christian")
display(Math(eq_sans_AMR_Chr))

## RK_1 
flux_total_sans_AMR_RK1 = (1/dt*CFL_DIFF * (u_xplus1 - 2*u(x,t) + u_xmoins1)).expand()
u_t_RK1 = - reste_serie_en_temps + flux_total_sans_AMR_RK1
u_t_tilde_RK1 = u_t_RK1.expand()
u_t_tilde_RK1 = apply_cauchy_kovaleskaya(u_t_tilde_RK1,u,x,t,D,ORDER_SERIES)
second_membre_RK1 = rendreJoli(u_t_tilde_RK1,u,x,t,dx,dt,ORDER_SERIES)
eq_sans_AMR_RK1 = f"{sp.latex(derivee_temporelle)} = {sp.latex(u_t_tilde_RK1)}"
print("Eq modifée temps-espace RKE1")
display(Math(eq_sans_AMR_RK1))

Eq modifée temps-espace RKE2


<IPython.core.display.Math object>

\frac{\partial}{\partial t} u{\left(x,t \right)} = - \frac{D^{4} \Delta t^{3} \frac{\partial^{8}}{\partial x^{8}} u{\left(x,t \right)}}{24} - \frac{D^{3} \Delta t^{2} \frac{\partial^{6}}{\partial x^{6}} u{\left(x,t \right)}}{6} + \frac{D \Delta x^{2} \frac{\partial^{4}}{\partial x^{4}} u{\left(x,t \right)}}{12} + D \frac{\partial^{2}}{\partial x^{2}} u{\left(x,t \right)}
Eq modifée temps-espace Version Christian


<IPython.core.display.Math object>

Eq modifée temps-espace RKE1


<IPython.core.display.Math object>

# Introduction of MRA

In [ ]:
s_hat = sp.Symbol('$\hat s$', positive=True)
deltal = sp.Symbol(u'\\Delta l', positive=True)

dictionnary             = computeEqModif(u,x,t,deltal,dx,dt,CFL_DIFF,ORDER_SERIES,stencil=1,DEBUG=DEBUG)
lhs,rhs,tex             = dictionnary["RK2"]
lhsRK1,rhsRK1,texRK1    = dictionnary["RK1"]
#if CHRISTIAN_VERSION : lhsChristain,rhsChristain,texChristain    = dictionnary["Christian"]
print("🗺️------------------------------------EQUA. MODIFIEES------------------------------------")
print("====================================\n✅🥋-AVEC AMR. \n====================================")
print("--->RK2")
display(Math(tex))
print(tex)
# if CHRISTIAN_VERSION : 
#     print("---> Version Christian")
#     display(Math(texChristain))
print("--->RK1")
display(Math(texRK1))
print("====================================\n❌🥋-SANS AMR. \n====================================")
display(Math(eq_sans_AMR))
print("====================================\n⚔️⚔️-DIFFERENCE\n====================================")
afficheSimpy((rhs - u_t_tilde).collect(
    [dx**2,
     dt*D**2*sp.diff(u(x,t),x,4)])
    )
diffDic = sp.collect( (rhs - u_t_tilde).expand() , [ dx**n for n in range(ORDER_SERIES*2,0,-1)] ,evaluate=False)
for (key,value) in diffDic.items():
    print("Différence d'ordre:")
    afficheSimpy(key)
    afficheSimpy(value.collect([dt *D**2*sp.diff(u(x,t), x ,ordreLocal) for ordreLocal in range(3*ORDER_SERIES)]))

🗺️------------------------------------EQUA. MODIFIEES------------------------------------
✅🥋-AVEC AMR. 
--->RK2


<IPython.core.display.Math object>

\frac{\partial}{\partial t} u{\left(x,t \right)}=\frac{2^{2 \Delta l} D^{2} \Delta t \frac{\partial^{4}}{\partial x^{4}} u{\left(x,t \right)}}{2} - \frac{2^{2 \Delta l} D \Delta l \Delta x^{2} \frac{\partial^{4}}{\partial x^{4}} u{\left(x,t \right)}}{4} + \frac{2^{2 \Delta l} D \Delta x^{2} \frac{\partial^{4}}{\partial x^{4}} u{\left(x,t \right)}}{12} + D \frac{\partial^{2}}{\partial x^{2}} u{\left(x,t \right)} - \frac{\Delta t^{3} \frac{\partial^{4}}{\partial t^{4}} u{\left(x,t \right)}}{24} - \frac{\Delta t^{2} \frac{\partial^{3}}{\partial t^{3}} u{\left(x,t \right)}}{6} - \frac{\Delta t \frac{\partial^{2}}{\partial t^{2}} u{\left(x,t \right)}}{2}
--->RK1


<IPython.core.display.Math object>

❌🥋-SANS AMR. 


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Différence d'ordre:


<IPython.core.display.Math object>

<IPython.core.display.Math object>